In [9]:
import pandas as pd

import os

In [10]:
netflow_filtered = pd.read_csv('pch/netflow/pch.20230329-20230331.csv')
netflow_filtered[["ts", "te"]] = netflow_filtered[["ts", "te"]].astype(dtype='datetime64[ns]')

用 sa grouping

In [11]:
grouped = netflow_filtered.groupby('sa')

for key, df in grouped:
    print("Key: " + key, " " * 20, end="\r")
    if ":" in key:
        continue
    targetFile = "pch/20230329-20230331/src_feature/" + key + ".parquet"
    if os.path.isfile(targetFile):
        continue
    
    temp = {
        "ts": [],
        "sa": [],
        "ipkt": [],
        "ibyt": [],
        "opkt": [],
        "obyt": [],
        "flows": [],
        "nda": [],
        "nsp": [],
        "ndp": [],
    }
    for row in df.itertuples():
        duration = int(row.td) + 1
        for t in [row.ts + pd.Timedelta(seconds=s) for s in range(duration)]:
            temp["ts"].append(t)
            temp["sa"].append(row.sa)
            temp["ipkt"].append(row.ipkt / duration)
            temp["ibyt"].append(row.ibyt / duration)
            temp["opkt"].append(row.opkt / duration)
            temp["obyt"].append(row.obyt / duration)
            temp["flows"].append(1)
            temp["nda"].append(1)
            temp["nsp"].append(1)
            temp["ndp"].append(1)
    temp = pd.DataFrame(temp)
    
    feature = pd.DataFrame()
    feature["timeStart"] = temp["ts"]
    feature["srcIP"] = temp["sa"]
    feature["packets"] = temp["ipkt"] + temp["opkt"]
    feature["bytes"] = temp["ibyt"] + temp["obyt"]
    feature["bytes/packets"] = feature["bytes"] / feature["packets"]
    feature["flows"] = temp["flows"]
    feature["flows/(bytes/packets)"] = feature["flows"] / feature["bytes/packets"]
    feature["nDstIP"] = temp["nda"]
    feature["nSrcPort"] = temp["nsp"]
    feature["nDstPort"] = temp["ndp"]
    os.makedirs(os.path.dirname(targetFile), exist_ok=True)
    feature.to_parquet(targetFile, index=False)

用 da grouping

In [12]:
grouped = netflow_filtered.groupby('da')

for key, df in grouped:
    print("Key: " + key, " " * 20, end="\r")
    if ":" in key:
        continue
    targetFile = "pch/20230329-20230331/dst_feature/" + key + ".parguet"
    if os.path.isfile(targetFile):
        continue
    
    temp = {
        "ts": [],
        "da": [],
        "ipkt": [],
        "ibyt": [],
        "opkt": [],
        "obyt": [],
        "flows": [],
        "nsa": [],
        "nsp": [],
        "ndp": [],
    }
    for row in df.itertuples():
        duration = int(row.td) + 1
        for t in [row.ts + pd.Timedelta(seconds=s) for s in range(duration)]:
            temp["ts"].append(t)
            temp["da"].append(row.da)
            temp["ipkt"].append(row.ipkt / duration)
            temp["ibyt"].append(row.ibyt / duration)
            temp["opkt"].append(row.opkt / duration)
            temp["obyt"].append(row.obyt / duration)
            temp["flows"].append(1)
            temp["nsa"].append(1)
            temp["nsp"].append(1)
            temp["ndp"].append(1)
    temp = pd.DataFrame(temp)
    
    feature = pd.DataFrame()
    feature["timeStart"] = temp["ts"]
    feature["dstIP"] = temp["da"]
    feature["packets"] = temp["ipkt"] + temp["opkt"]
    feature["bytes"] = temp["ibyt"] + temp["obyt"]
    feature["bytes/packets"] = feature["bytes"] / feature["packets"]
    feature["flows"] = temp["flows"]
    feature["flows/(bytes/packets)"] = feature["flows"] / feature["bytes/packets"]
    feature["nSrcIP"] = temp["nsa"]
    feature["nSrcPort"] = temp["nsp"]
    feature["nDstPort"] = temp["ndp"]
    os.makedirs(os.path.dirname(targetFile), exist_ok=True)
    feature.to_parquet(targetFile, index=False)